In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import time
from tqdm import tqdm

In [2]:
import openpyxl
from openpyxl.styles import PatternFill,Color
from openpyxl import Workbook
from random import *

## Edge에 옵션을 주는 방법, 그리고 Headless
- pip install msedge-selenium-tools selenium==3.141 설치 필요

### headless 옵션을 사용할 땐 속도는 빠르지만 창을 띄우지 않기 때문에 driver의 click() 이벤트를 execute할 수 없다 <br>
=> Message: element not interactable 오류 발생 <br>
해결: options.add_argument('--window-size=1920x1080')




In [46]:
from msedge.selenium_tools import Edge, EdgeOptions

In [47]:
options = EdgeOptions()
options.use_chromium = True 
options.add_argument("headless")
options.add_argument("disable-gpu")

# Message: element not interactable 오류 방지
options.add_argument('--window-size=1920x1080')

# edge 드라이버를 직접 다운로드 받을 때의 방법
driver = Edge('./msedgedriver.exe', options=options)

In [48]:
# 크롤링할 단어
# space는 +로 바꿈
def get_keyword(text):
    return text.replace(" ","%20")

In [49]:
# 정렬 우선순위
def sort_kind(index):
    if index == 1:
        return 'vcount' # 추천순
    elif index == 2:
        return 'date' # 최신순
    else:
        return 'none' # 정확도순

In [50]:
keyword = '여친 선물'
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword) )

# Headless 제대로 동작하나요? => 스크린샷
driver.get_screenshot_as_file('./tmp/naver_main.png')


# random.uniform(a,b): a~b까지 동일 확률로 하나의 수
time.sleep(uniform(0.1, 1.0))

## 기간 설정
- 최근 3일 간

In [51]:
page_index = 1
start = datetime.now() + timedelta(days=-3)
start =datetime.strftime(start,'%Y.%m.%d')
print(start, type(start))

end = datetime.strftime(datetime.now(),'%Y.%m.%d')
print(end, type(end))

period = '&period=' + start + '.%7C' + end + '.'
print(period)

2020.12.06 <class 'str'>
2020.12.09 <class 'str'>
&period=2020.12.06.%7C2020.12.09.


In [52]:
today = end
today

'2020.12.09'

In [53]:
# url 저장
f = open('./result/url_list_' + get_keyword(keyword) + '_' + today + '.txt', 'w')
f

<_io.TextIOWrapper name='./result/url_list_여친%20선물_2020.12.09.txt' mode='w' encoding='cp949'>

In [54]:
page_url = []
while True:
    time.sleep(uniform(0.1, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + sort_kind(0) + '&query=' + get_keyword(keyword) + period + "&section=kin" + "&page=" + str(page_index))

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.select('ul.basic1 li > dl > dt > a')
    for tag in tags:
        url = tag.get('href')
        page_url.append(url)
        f.write(url + '\n')
    
    post_number = soup.select_one('.number').text.lstrip('(').rstrip(')').replace(',','')
    current_number = int(post_number.split('/')[0].split('-')[1])
    total_number = int(post_number.split('/')[1])

    print('현재 글 수:',current_number)   
    if current_number == total_number:
        break
    else:
        page_index += 1

print('총 글 수:',total_number)
f.close()

현재 글 수: 7
총 글 수: 7


- url 파일 한 줄씩 읽어 리스트로 만들기

In [55]:
with open('./result/url_list_' + get_keyword(keyword) + '_' + today + '.txt', 'r') as f:
    list_file = f.readlines()
list_file = [line.rstrip('\n') for line in list_file] 

In [56]:
print(type(list_file))
list_file[0]

<class 'list'>


'https://kin.naver.com/qna/detail.nhn?d1id=6&dirId=6130202&docId=375046270&qb=7Jes7LmcIOyEoOusvA==&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

- 엑셀로 저장할 때 파일 이름

In [57]:
excelname = './result/' + get_keyword(keyword) + '_' + today + '_crawling_result.xlsx'
excelname

'./result/여친%20선물_2020.12.09_crawling_result.xlsx'

In [58]:
wb = Workbook()
sheet = wb.active
sheet.append(['제목','질문','답변'])

In [62]:
for j in range(1, 4):
    sheet.cell(row = 1, column=j).fill = PatternFill(start_color='808080', end_color='808080', fill_type='solid')

for url in tqdm(list_file):

    driver.get(url)
    # html = requests.get(url).text
    # soup = BeautifulSoup(html, 'html.parser')

    # 답변 개수 때문에 bs4 대신 selenium 사용
    answer_count = int(driver.find_element_by_css_selector('em._answerCount.num').text)
    print('\n tag로 가져온 글 개수:',answer_count)
    for i in range( (answer_count-1) // 5 ):
        more_5 = driver.find_element_by_id('nextPageButton')
        more_5.click()
        time.sleep(uniform(0.1, 1.0))

    # tag로 들어오는 글 개수와 실제 들어오는 글 개수가 달랐던 이유
    # 클릭을 완료한 뒤에 page_source를 가져와야 제대로 들어오는데 이 코드가 위에 가 있었음
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # main 제목
    title = soup.select_one('div.title').text.lstrip('"').rstrip('"').strip()

    try:
        content_text = soup.select_one('div.c-heading__content').text.strip()
    except:
        content_text = ""
    
    answer_lists = soup.select("div._endContentsText.c-heading-answer__content-user")
    print('실제 들어오는 글 개수:',len(answer_lists))

    for i, answer in enumerate(answer_lists):
        lines = answer.select('span')
        answer_text = ""
        for line in lines:
            answer_text += line.text.strip()
        
        if i == 0:
            sheet.append([title,content_text,answer_text])
        else:
            sheet.append(["","",answer_text])
    wb.save(excelname)



  0%|          | 0/7 [00:00<?, ?it/s]
 tag로 가져온 글 개수: 6
실제 들어오는 글 개수: 6


 14%|█▍        | 1/7 [00:05<00:32,  5.48s/it]
 tag로 가져온 글 개수: 6
실제 들어오는 글 개수: 6


 29%|██▊       | 2/7 [00:08<00:23,  4.73s/it]
 tag로 가져온 글 개수: 9
실제 들어오는 글 개수: 8


 43%|████▎     | 3/7 [00:11<00:17,  4.27s/it]
 tag로 가져온 글 개수: 5
실제 들어오는 글 개수: 5


 57%|█████▋    | 4/7 [00:15<00:11,  4.00s/it]
 tag로 가져온 글 개수: 5
실제 들어오는 글 개수: 5


 71%|███████▏  | 5/7 [00:17<00:07,  3.51s/it]
 tag로 가져온 글 개수: 8
실제 들어오는 글 개수: 7


 86%|████████▌ | 6/7 [00:20<00:03,  3.29s/it]
 tag로 가져온 글 개수: 5
실제 들어오는 글 개수: 5


100%|██████████| 7/7 [00:24<00:00,  3.48s/it]


In [42]:
print(6)
for i in range((6-1) // 5):
    print('몇번')

6
몇번


### 삭제되었거나, 글은 없고 사진만 있는 경우에는 tag 글 개수와 실제 글 개수에 차이가 있을 수 있는 것으로 보임